<a href="https://colab.research.google.com/github/Jezandre/Python/blob/main/Projeto_Recomenda%C3%A7%C3%A3o_filmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introdução**

Sistema de recomendações estão cada vez mais comuns no nosso dia a dia, desde compras até qual filme pretendemos assistir na netflix. Tais sistemas possuem grandes aplicações no setor de vendas pois auxilia o consumidor a adquirir produtos baseado nas informações fornecidas.

Muita das vezes aquele produto que você nem pensou que precisava aparece ali no processo de compra simplesmente porque você adquiriu algo parecido. Muitos destes precessos utilizam  algoritimos de TF-IDF (Term Frequency-Inverse Document Frequency), que nada mais é que a avaliação de termos parecidos em uma descrição e comparação com outros produtos.

**Objetivo**

O presente artigo é apenas uma introdução a esse mundo, demosntrar e descrever pois faz parte do meu processo de aprendizagem. Uma maneira de enxergar um pouco como funciona um sistema de recomendações baseadas na descrição.

Os dataset utilizados estão disponíveis na plataforma kaggle no  e são informações de filmes. O objetivo é indicar filmes baseados nas descrições fornecidas na sinopse.

Utilizei o seguinte vídeo como guia para a elaboração https://www.youtube.com/watch?v=ijtxuF_5kEU. Vale a pena ressaltar que o processo de criação de recomendações baseados na descrição é apenas um passo, outros passos precisam estar associados para que as recomendações possam ser ainda mais precisas.

**Passo a Passo**

Para iniciar primeiro precisamos fazer o download dos datasets disponíveis no link: https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

A implementação do código utilizei o google Colab pois é uma IDE bem simples de trabalhar e possui muitas bibliotecas do Python já instalados.

Como código de programação utilizei o Python 3.10.12

In [2]:
!python --version

Python 3.10.12


**Importando Bibliotecas**

As bibliotecas utilizadas serão pandas para criação de dataframes, numpy para tratamento dos dados. TfidfVectorizer para vetorização de texto que é uma das principais etapas do processamento de linguagem natural. linear_kernel que será utilizado para o cálculo de produto escalar dos vetores criados.

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

**Importando datasets**

No googlecolab você pode utilizar a pasta no canto esquerda para fazer o upload dos seus arquivos e utilizar o endereço a partir da localização dele para importar os seus dataframes. A importação é feita utilizando a biblioteca pandas seguido dos seguintes comandos e localização do arquivo.



In [6]:
df_movies = pd.read_csv('/content/tmdb_5000_movies.csv')


**Analise do dataset**

É importante sempre que importar um arquivo para o dataframe utilizar a função head para visualizar as colunas.

In [7]:
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2.787965e+09,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800.0
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,9.610000e+08,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500.0
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,8.806746e+08,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466.0
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1.084939e+09,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106.0
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,2.841391e+08,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124.0


Assim a parte que precisamos do dataset para utilizar o sistema de recomendação é a coluna "*overview*" pois é nela que temos os dados de sinopse que utilizaremos para criar o nosso sistema de recomendação.

In [8]:
df_movies['overview']

0       In the 22nd century, a paraplegic Marine is di...
1       Captain Barbossa, long believed to be dead, ha...
2       A cryptic message from Bond’s past sends him o...
3       Following the death of District Attorney Harve...
4       John Carter is a war-weary, former military ca...
                              ...                        
2430    Evil Dead, the fourth installment of the Evil ...
2431    A Greek tour guide named Georgia attempts to r...
2432    The new season of "American Dreamz," the wildl...
2433    With global superpowers engaged in an increasi...
2434    After the death of Raya Green's(Rutina Wesley)...
Name: overview, Length: 2435, dtype: object

**Tratamento dos dados**

Como qualquer dataset que você for utilizar para análise ou ciencia de dados o tratamento de dados é uma das etapas mais importantes, é preciso garantir que as informações estejam adequadas para a utilização de algorítiomos de AI.

Para o nosso estudo aqui utilizaremos a remoção de stopwords. O que seria isso? Stop words são palavras de ligação que muita das fezes não tem uma relevância grande para a informação neste caso. Seriam as preposições, artigos, etc.

Nesse caso TfidfVectorizer possui uma função para a remoção dessas palavras em vários idiomas (Não vou saber dizer ao certo quais idiomas).

No caso do nosso dataset estamos utilizando a base em inglês então o parâmetro a ser utilizado é "*english*".

Precisamos tambem remover os valores nulo do nosso data set e para isso utilizamos o fillna("") que vai substituir estes valores por  vazio.

E por fim para aplicarmos ao nosso dataset a remoção de *stopwords* utilizamos a função fit_transform, assim teremos uma base apenas com as palavras mais relevantes para nosso contexto.



In [9]:
tfidf = TfidfVectorizer(stop_words="english")
df_movies['overview'] = df_movies['overview'].fillna("")
tfidf_matrix = tfidf.fit_transform(df_movies['overview'])

 **Calculando o produto vetorial**

 Em seguida utiliza-se o linear_kernel para calcular o produto vetorial da seguinte maneira. Esse cáculo é fundamental pois é a partir dele que vamos descobrir a importancia relativa de uma palavra dentro da descrição do filme. O processo transforma valores textuais em números de forma que a máquina possa "compreender" melhor e trazer informções mais acertivas

In [10]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

**Indices**

Em seguida é preciso remover os ídices duplicados do dataset. Índices duplicados podem causar erros de relacionamento e trazer informações erradas.

In [11]:
indices = pd.Series(df_movies.index, index=df_movies['original_title']).drop_duplicates()
indices

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
Evil Dead                                   2430
My Life in Ruins                            2431
American Dreamz                             2432
Superman IV: The Quest for Peace            2433
How She Move                                2434
Length: 2435, dtype: int64

In [12]:
indices['The Dark Knight Rises']

3

**Criando função de recomendação**

A função criada a seguir recebe dois parâmetros, o "*title*" título que é o ítem para o qual você quer saber quais files serão recomendados e "*consine_sim*" que é um parametro opcional.

Quando temos esta expressão nos argumentos da função:

"cosine_sim = cosine_sim"

significa que se você já calculou ele anteriormente o valor pode ser utilizado.

O idx vai obter o índice do título a ser pesquisado no dataframe

idx = indice[title]

sim_scores avalia a similaridade do filme em relação ao filme que foi selecionado e cria uma lista com os índices que posteriormente será relacionado com os dados do filme no dataframe através da função iloc. E pronto temos um sistema simples de recomendação baseado na descrição da sinopse.



In [19]:
def get_recommendations(title, cosine_sim = cosine_sim):
  idx = indices[title]
  sim_scores = enumerate(cosine_sim[idx])
  sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
  sim_scores = sim_scores[1:11]
  cosine_sim[idx]
  sim_index = [i[0] for i in sim_scores]
  print(sim_index)
  print(df_movies[ 'original_title'].iloc[sim_index])


get_recommendations('Spectre')

[1343, 1717, 11, 29, 1880, 1131, 1743, 1200, 698, 2238]
1343    Never Say Never Again
1717               Safe Haven
11          Quantum of Solace
29                    Skyfall
1880              Dance Flick
1131          Licence to Kill
1743                Octopussy
1200     The Living Daylights
698       The Prince of Egypt
2238                 Bandslam
Name: original_title, dtype: object



**Conclusão**

Como podemos ver o sistema desenvolvido em questão é uma das mais simples aplicações para recomendação. Existem diversos algorítmos bem mais robustos que podem trazer muito mais resultados. Porém explorar os conceitos básicos são primordiais para o desenvolvimento de aplicações mais robustas. Quem nunca se deparou nos estudos e no trabalho se lembrando de um conceito fundamental que pode ser aplicado na prática não é mesmo.

Sujestões de aplicação correções estejam livres em colocar nos comentários.

